<a href="https://colab.research.google.com/github/saivenkatakrishna2/Fine-tuning-Object-Detection/blob/main/SLM_vs_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import time
import torch
import psutil
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [12]:
def run_model(model_name, user_input):
    print(f"\nLoading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cpu")
    summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    prompt = f"summarize: {user_input}"

    # Token count before inference
    input_token_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_token_count = input_token_ids.shape[1]

    # Time and memory measurement
    start_time = time.time()
    mem_before = psutil.Process().memory_info().rss / 1024 / 1024

    result = summarizer(prompt, max_length=100, min_length=20, do_sample=False)

    mem_after = psutil.Process().memory_info().rss / 1024 / 1024
    end_time = time.time()

    summary_text = result[0]['generated_text']
    output_token_ids = tokenizer.encode(summary_text, return_tensors="pt")
    output_token_count = output_token_ids.shape[1]
    sentence_count = summary_text.count('.')

    return {
        "output": summary_text,
        "time_taken_sec": round(end_time - start_time, 2),
        "memory_used_mb": round(mem_after - mem_before, 2),
        "input_token_count": input_token_count,
        "output_token_count": output_token_count,
        "summary_sentence_count": sentence_count,
    }

In [13]:
# === Get user input ===
user_input = input("Enter the paragraph you want to summarize:\n")


Enter the paragraph you want to summarize:
Cryptocurrency is a digital or virtual currency secured by cryptography, existing only online and not tied to a physical form like cash or coins. It operates on a decentralized system, meaning no central authority like a bank or government controls it. Instead, transactions are recorded on a public ledger called a blockchain, and cryptography verifies and secures these transactions. 


In [14]:
# === Define models ===
slm_model = "google/flan-t5-small"   # Small Language Model (~80M params)
llm_model = "google/flan-t5-xl"      # Large Language Model (~3B params)



In [15]:
# === Run SLM ===
print("\n--- SLM: google/flan-t5-small ---")
slm_result = run_model(slm_model, user_input)
print("Summary:", slm_result["output"])
print("Time (s):", slm_result["time_taken_sec"])
print("Memory Used (MB):", slm_result["memory_used_mb"])
print("Input Tokens:", slm_result["input_token_count"])
print("Output Tokens:", slm_result["output_token_count"])
print("Sentences in Summary:", slm_result["summary_sentence_count"])


--- SLM: google/flan-t5-small ---

Loading model: google/flan-t5-small


Device set to use cpu


Summary: Using cryptography to secure transactions, the cryptocurrency is a digital or virtual currency. Using cryptography to secure transactions, the cryptocurrency is a digital or virtual currency.
Time (s): 2.49
Memory Used (MB): 222.37
Input Tokens: 85
Output Tokens: 37
Sentences in Summary: 2


In [16]:
# === Run LLM ===
print("\n--- LLM: google/flan-t5-xl ---")
llm_result = run_model(llm_model, user_input)
print("Summary:", llm_result["output"])
print("Time (s):", llm_result["time_taken_sec"])
print("Memory Used (MB):", llm_result["memory_used_mb"])
print("Input Tokens:", llm_result["input_token_count"])
print("Output Tokens:", llm_result["output_token_count"])
print("Sentences in Summary:", llm_result["summary_sentence_count"])


--- LLM: google/flan-t5-xl ---

Loading model: google/flan-t5-xl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Summary: Understand how cryptocurrency works. Understand how blockchain works. Understand how cryptography works. Understand how blockchain works.
Time (s): 61.79
Memory Used (MB): 8265.41
Input Tokens: 85
Output Tokens: 22
Sentences in Summary: 4
